In [25]:
from __future__ import division
import os.path
import numpy as np
import tqdm # task bar
import matplotlib.pyplot as plt
%matplotlib notebook
import idx2numpy as inp
import scipy.sparse.linalg
import scipy.spatial.distance as dist
from sklearn import svm

from cvxopt import matrix
from cvxopt import solvers

# Import Data

In [3]:
TrImgs = np.array([img.flatten() for img in inp.convert_from_file('train-images.idx3-ubyte')])
TrLbls = inp.convert_from_file('train-labels.idx1-ubyte')
TsLbls = inp.convert_from_file('t10k-labels.idx1-ubyte')

In [4]:
[TrImgs40Comps,TrImgs80Comps,TrImgs200Comps] = [np.load('TrImgs{}Comps.npy'.format(c)) for c in [40,80,200]]
[TsImgs40Comps,TsImgs80Comps,TsImgs200Comps] = [np.load('TsImgs{}Comps.npy'.format(c)) for c in [40,80,200]]

## Test with Scikitlearn package to check expected accuracy

In [5]:
clf = svm.LinearSVC()
clf.fit(TrImgs40Comps[:1000],TrLbls[:1000])

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [6]:
print 'Approx Fit Accuracy = {} %'.format(np.sum([clf.predict([TrImgs40Comps[2000+i]])==TrLbls[2000+i] for i in np.arange(40)])/40*100)

Approx Fit Accuracy = 72.5 %


# Build QP 
Minimise over X: 0.5 X.P.X + q.X (cvxopt notation)

Minimise over W,b,{e}
under constraints A,B (see below)

where X = [W,b,{e}]
* W: D-element vector
* b: scalar
* {e}: N-element vector
Hence

* P = Identity(D+1) subspace
* q = c * Identity(N) subspace

## Cast Constraints into array
takes Training Images and Labels, and Constraint (constraint parameter) as input 
and casts the constraint equations:
* yi * (w.xi + b) + ei >= 1 (Constraint A: N of them)
* ei >= 0 (Constraint B: N of them)

which can be expressed in a giant matrix inequality GX <= h acceptable to cvxopt (QP library):
* G: Constraint Array = -[[Ga, Gb],[Gc, Gd]] (shape = 2N by D+1+N)
    * Ga = [y1[x1, 1],y2[x2, 1],...,yN[xN, 1]], where x1 is D elements long (shape = N by D+1) 
    * Gb = Identity(N) (shape = N by N)
    * Gc = Zero(N) (shape = N by D+1)
    * Gd = Identity(N) 
* X: [w1,w2,...,wD,b,e1,e2,...,eN] (D+1+N)
* h: -[1,....,1,0,...,0] (N x 1's and N x 0's)
* the negative coefficient is to convert the original '>=' AB constraints into '<=' constraints

For each class, we build a G.

In [147]:
def QP_params(Imgs,Lbls,Class,ConstraintParameter):
    """
    Builds Quadratic Programming Parameters
    """
    BinLbls = (-1)**(np.array([Lbls==Class]).astype('int')+1).flatten()
    N = len(Lbls)
    D = len(Imgs[0])
    """Builds G, the constraint array for training linear SVM for specified class Class"""
    yx = [BinLbls[i] * Imgs[i] for i in np.arange(N)]
    Ga = np.c_[yx,BinLbls]
    Gb = np.identity(N)
    Gc = np.zeros((N,D+1))
    Gd = np.identity(N)
    G = -np.r_[np.c_[Ga,Gb],np.c_[Gc,Gd]]
    """Build h"""
    h = -np.r_[np.ones(N),np.zeros(N)]
    """Build P, q"""
    P = np.r_[np.c_[np.identity(D+1),np.zeros((D+1,N))],np.zeros((N,D+1+N))]
    q = ConstraintParameter * \
    np.r_[np.zeros(D+1),np.ones(N)]
    return P, q, G, h

In [21]:
p,q,g,h = QP_params(TrImgs40Comps[:5],TrLbls[:5],1,1)

(5,)


In [78]:
def FitSVM(Imgs,Lbls,Class,ConstraintParameter):
    N = len(Lbls)
    D = len(Imgs[0])
    P,q,G,h = QP_params(Imgs,Lbls,Class,ConstraintParameter)
    P = matrix(P)
    q = matrix(q)
    G = matrix(G)
    h = matrix(h)
    sol = solvers.qp(P,q,G,h)
    X = np.array(sol['x'])
    w = X[:D] # up to and includes index D-1 (the D-th element)
    b = X[D] # index D (the D+1 th element)
    e = X[-N:]
    return w, b, e

In [135]:
np.r_[w.flatten(),b,e.flatten()].reshape

(5785,)

In [145]:
def FitAllSVM(Imgs,Lbls,p,ConstraintParameter):
    """Fit Data for All Classes"""
    f = open('SVM_fit_params_{}_components_{}_constraint.dat'.format(p,ConstraintParameter),'a')
    classes = np.unique(Lbls)
    for Class in classes:
        print('Fitting Class {}\n'.format(Class))
        w,b,e = FitSVM(Imgs,Lbls,Class,ConstraintParameter)
        X = np.r_[w.flatten(),b,e.flatten()]
        L = len(X)
        X = X.reshape(1,L)
        np.savetxt(f,X)
    f.close()

In [146]:
FitAllSVM(TrImgs40Comps,TrLbls,40,1e-2)

Fitting Class 0

(100,)
     pcost       dcost       gap    pres   dres
 0: -9.9774e+01  1.0111e+02  4e+02  3e+00  1e+04
 1:  2.3025e+01 -5.4197e+00  3e+01  7e-02  2e+02
 2:  4.4969e+00 -9.5332e-01  6e+00  1e-02  4e+01
 3:  5.0411e-01 -9.9026e-02  7e-01  1e-03  4e+00
 4:  2.7600e-02 -5.4156e-03  4e-02  6e-05  2e-01
 5:  3.3763e-03 -6.2516e-04  4e-03  6e-06  2e-02
 6:  2.0304e-04 -2.7603e-05  2e-04  2e-07  8e-04
 7:  2.1996e-05  1.0039e-05  1e-05  2e-09  8e-06
 8:  1.7180e-05  1.3192e-05  4e-06  7e-10  2e-06
 9:  1.6407e-05  1.4874e-05  2e-06  4e-11  1e-07
10:  1.5501e-05  1.5318e-05  2e-07  4e-12  1e-08
11:  1.5392e-05  1.5382e-05  1e-08  3e-14  1e-10
Optimal solution found.
885
Fitting Class 1

(100,)
     pcost       dcost       gap    pres   dres
 0: -9.9493e+01  1.0172e+02  4e+02  3e+00  1e+04
 1:  2.2763e+01 -7.0708e+00  4e+01  9e-02  4e+02
 2:  5.9160e+00 -1.6075e+00  9e+00  2e-02  8e+01
 3:  9.5639e-01 -2.4818e-01  1e+00  2e-03  1e+01
 4:  3.2155e-02 -8.3910e-03  5e-02  8e-05  3

In [140]:
r = np.loadtxt('SVM_fit_params_40_components_1_constraint.dat')

In [144]:
r[0].shape

(885,)

In [113]:
for i in np.arange(100):
    if np.dot(w.T,TsImgs40Comps[i])+b >= 1:
        print i, np.dot(w.T,TsImgs40Comps[i])+b

33 [ 1.65272865]
35 [ 5.01032753]
64 [ 3.12056477]
72 [ 1.49682933]
77 [ 2.19805742]
82 [ 9.51295666]


In [112]:
for i in np.arange(100):
    if TsLbls[i] == 2:
        print i

1
35
38
43
47
72
77
82


In [130]:
f=open('asd.dat','a')
for iind in range(4):
    a=np.random.rand(2).reshape(1,2)
    np.savetxt(f,a)
f.close()

In [128]:
np.random.rand(1,2)

array([[ 0.57855072,  0.82239132]])